## TUGAS BESAR KULIAH SISTEM PENGUKURAN BERBASIS CITRA
Nama :  Ronggur Mahendra Widya Putra

NIM  :  13519008

In [1]:
# Import Library
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import cv2 
import os
import numpy as np
import scipy
from skimage import color, data, restoration
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from random import uniform
from tensorflow.keras.layers import BatchNormalization

# Loading Data dari folder ./Data/ClassifiedData

In [2]:

ClassifiedDataDir = "./Data/ClassifiedData/Train"
BATCH_SIZE = 32 
IMG_SIZE = (128, 128)
train_dataset = tf.keras.utils.image_dataset_from_directory(
    directory = ClassifiedDataDir,
        labels='inferred',
        label_mode='int',
        class_names=None,
        color_mode='rgb',
        batch_size=BATCH_SIZE,
        image_size=IMG_SIZE,
        shuffle=True,
        seed=1234,
        subset="training",
        validation_split=0.1,
        interpolation='bilinear',
        follow_links=False,
        crop_to_aspect_ratio=False,
        # rescale = 1./255,
    )
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    directory = ClassifiedDataDir,
        labels='inferred',
        label_mode='int',
        class_names=None,
        color_mode='rgb',
        batch_size=BATCH_SIZE,
        image_size=IMG_SIZE,
        shuffle=True,
        seed=1234,
        subset="validation",
        validation_split=0.1,
        interpolation='bilinear',
        follow_links=False,
        crop_to_aspect_ratio=False,
        # rescale = 1./255,
    )

    
test_dataset = tf.keras.utils.image_dataset_from_directory(
    directory = "./Data/ClassifiedData/Test",
        labels='inferred',
        label_mode='int',
        class_names=None,
        color_mode='rgb',
        batch_size=BATCH_SIZE,
        image_size=IMG_SIZE,
        shuffle=True,
        seed=None,
        interpolation='bilinear',
        follow_links=False,
        crop_to_aspect_ratio=False,
        # rescale = 1./255,
    )

Found 3088 files belonging to 24 classes.
Using 2780 files for training.
Found 3088 files belonging to 24 classes.
Using 308 files for validation.
Found 312 files belonging to 24 classes.


List Class

In [3]:
# for element in dataset.as_numpy_iterator():
#     print(element)
class_names = train_dataset.class_names
print(class_names)
print(len(class_names))

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']
24


In [4]:
# print('Number of training batches: %d' % tf.data.experimental.cardinality(train_dataset).numpy())
# print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset).numpy())

# Optimization buffer untuk Dataset

In [5]:
# DataSet
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

# Makingsure Training using GPU

In [6]:
print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices('GPU')) 
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs Available:  1


# Creating CNN Model 

## Model 1

In [7]:
# # Model 1 Initialize
input_size = 128
filter_size = 14
num_filter = 8
maxpool_size = 2
batch_size = BATCH_SIZE
epochs = 30

model_1 = models.Sequential()
model_1.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(input_size, input_size, 3)))
model_1.add(layers.MaxPooling2D((2, 2)))
model_1.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_1.add(layers.MaxPooling2D((2, 2)))
model_1.add(layers.Conv2D(64, (3, 3), activation='relu'))

model_1.add(tf.keras.layers.Flatten())
model_1.add(tf.keras.layers.Dense(120))
model_1.add(tf.keras.layers.Dense(24, activation = 'softmax'))

model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 50176)             0

In [8]:
# Optimizer
optimizer1 = tf.keras.optimizers.Nadam(
    learning_rate=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-07,
    name='Nadam'
) # 0.00001

# Loss Fn
lossfn1 = tf.keras.losses.SparseCategoricalCrossentropy( from_logits=False, reduction=tf.keras.losses.Reduction.AUTO, name='sparse_categorical_crossentropy')

# Model Summary
model_1.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)

## Model 2

In [9]:
# # Model 2

input_size = 128
filter_size = 3
num_filter = 8
maxpool_size = 2
batch_size = BATCH_SIZE
epochs = 30

steps_per_epoch = 24720/batch_size

model_2 = tf.keras.models.Sequential()
model_2.add(tf.keras.layers.Conv2D(16, (filter_size,filter_size), 
                 input_shape= (input_size,input_size,3), 
                 activation ='relu',
                 padding='same'))
model_2.add(tf.keras.layers.Conv2D(16, (filter_size,filter_size), 
                 input_shape= (input_size,input_size,3), 
                 activation ='relu',
                 padding='same'))
model_2.add(tf.keras.layers.MaxPooling2D(pool_size=(maxpool_size, maxpool_size),strides=1))
model_2.add(tf.keras.layers.Dropout(uniform(0, 1)))

model_2.add(tf.keras.layers.Conv2D(32, (filter_size,filter_size), 
                 activation='relu', 
                 padding='valid'))
model_2.add(tf.keras.layers.Conv2D(32, (filter_size,filter_size), 
                 activation='relu', 
                 padding='valid'))
model_2.add(BatchNormalization())
model_2.add(tf.keras.layers.MaxPooling2D(pool_size=(maxpool_size, maxpool_size),strides=2))
model_2.add(tf.keras.layers.Dropout(uniform(0, 1)))  

model_2.add(tf.keras.layers.Conv2D(32, (filter_size,filter_size), 
                 activation='relu', 
                 padding='valid'))
model_2.add(tf.keras.layers.Conv2D(32, (filter_size,filter_size), 
                 activation='relu', 
                 padding='valid'))
model_2.add(BatchNormalization())

model_2.add(tf.keras.layers.Conv2D(32, (filter_size,filter_size), 
                 activation='relu', 
                 padding='valid'))
model_2.add(tf.keras.layers.Conv2D(32, (filter_size,filter_size), 
                 activation='relu', 
                 padding='valid'))
model_2.add(BatchNormalization())
model_2.add(tf.keras.layers.MaxPooling2D(pool_size=(maxpool_size, maxpool_size),strides=2))
model_2.add(tf.keras.layers.Dropout(uniform(0, 1)))  

model_2.add(tf.keras.layers.Flatten())
model_2.add(tf.keras.layers.Dense(120, activation='relu'))
model_2.add(tf.keras.layers.Dense(120, activation='relu'))
model_2.add(tf.keras.layers.Dense(24,activation='softmax'))



In [10]:
METRICS = [ 'sparse_categorical_accuracy']
model_2.compile( optimizer= tf.keras.optimizers.Adam(lr=0.001),loss='sparse_categorical_crossentropy',metrics=METRICS)

model_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 128, 128, 16)      448       
                                                                 
 conv2d_4 (Conv2D)           (None, 128, 128, 16)      2320      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 127, 127, 16)     0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 127, 127, 16)      0         
                                                                 
 conv2d_5 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 conv2d_6 (Conv2D)           (None, 123, 123, 32)      9248      
                                                      

c:\Users\ZEPHYRUS GU502GU\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


# Model 3

In [11]:
#Model 3
input_size = 128
filter_size = 14
num_filter = 8
maxpool_size = 2
batch_size = BATCH_SIZE
epochs = 30

model_3 = tf.keras.models.Sequential()
model_3.add(tf.keras.layers.Conv2D(16, (filter_size,filter_size), 
                 input_shape= (input_size,input_size,3), 
                 activation ='relu',
                 padding='same'))
model_3.add(tf.keras.layers.Conv2D(32, (filter_size,filter_size), 
                 activation='relu', 
                 padding='valid'))
model_3.add(tf.keras.layers.MaxPooling2D(pool_size=(maxpool_size, maxpool_size),strides=1))

model_3.add(tf.keras.layers.Conv2D(32, (filter_size,filter_size), 
                 activation='relu', 
                 padding='valid'))
model_3.add(tf.keras.layers.Conv2D(32, (filter_size,filter_size), 
                 activation='relu', 
                 padding='valid'))
model_3.add(tf.keras.layers.MaxPooling2D(pool_size=(maxpool_size, maxpool_size),strides=2))
model_3.add(tf.keras.layers.Dropout(uniform(0, 1)))  


model_3.add(tf.keras.layers.Conv2D(64, (filter_size,filter_size), 
                 activation='relu', 
                 padding='valid'))
model_3.add(tf.keras.layers.Conv2D(64, (filter_size,filter_size), 
                 activation='relu', 
                 padding='valid'))
model_3.add(BatchNormalization())
model_3.add(tf.keras.layers.MaxPooling2D(pool_size=(maxpool_size, maxpool_size),strides=2))
model_3.add(tf.keras.layers.Dropout(uniform(0, 1)))  


model_3.add(tf.keras.layers.Flatten())
model_3.add(tf.keras.layers.Dense(128, activation='relu'))
model_3.add(tf.keras.layers.Dense(128, activation='relu'))
model_3.add(tf.keras.layers.Dense(24,activation='softmax'))

In [12]:
METRICS = [ 'sparse_categorical_accuracy']

model_3.compile( optimizer= tf.keras.optimizers.Adam(lr=0.001),loss='sparse_categorical_crossentropy',metrics=METRICS)

model_3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 128, 128, 16)      9424      
                                                                 
 conv2d_12 (Conv2D)          (None, 115, 115, 32)      100384    
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 114, 114, 32)     0         
 2D)                                                             
                                                                 
 conv2d_13 (Conv2D)          (None, 101, 101, 32)      200736    
                                                                 
 conv2d_14 (Conv2D)          (None, 88, 88, 32)        200736    
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 44, 44, 32)       0         
 2D)                                                  

# Training

In [13]:
# Training Model 1
history1 = model_1.fit(    
    train_dataset,
    epochs=30,
    shuffle=True,
    validation_data = (validation_dataset)
    )


Epoch 1/30
87/87 [==============================] - 8s 51ms/step - loss: 165.8931 - sparse_categorical_accuracy: 0.4658 - val_loss: 0.5191 - val_sparse_categorical_accuracy: 0.8864
Epoch 2/30
87/87 [==============================] - 4s 48ms/step - loss: 3.4740 - sparse_categorical_accuracy: 0.8590 - val_loss: 0.2727 - val_sparse_categorical_accuracy: 0.9318
Epoch 3/30
87/87 [==============================] - 5s 49ms/step - loss: 21.3086 - sparse_categorical_accuracy: 0.8338 - val_loss: 0.4336 - val_sparse_categorical_accuracy: 0.9675
Epoch 4/30
87/87 [==============================] - 5s 50ms/step - loss: 2.3258 - sparse_categorical_accuracy: 0.9421 - val_loss: 0.1463 - val_sparse_categorical_accuracy: 0.9870
Epoch 5/30
87/87 [==============================] - 5s 50ms/step - loss: 10.2403 - sparse_categorical_accuracy: 0.9428 - val_loss: 0.4589 - val_sparse_categorical_accuracy: 0.9903
Epoch 6/30
87/87 [==============================] - 5s 50ms/step - loss: 2.9069 - sparse_categorical_

In [14]:
# Training Model 2
history2 = model_2.fit(    
    train_dataset,
    epochs=30,
    shuffle=True,
    validation_data = (validation_dataset)
    )


Epoch 1/30
87/87 [==============================] - 16s 139ms/step - loss: 2.5268 - sparse_categorical_accuracy: 0.3691 - val_loss: 0.8988 - val_sparse_categorical_accuracy: 0.9123
Epoch 2/30
87/87 [==============================] - 10s 110ms/step - loss: 0.2476 - sparse_categorical_accuracy: 0.9342 - val_loss: 23.5757 - val_sparse_categorical_accuracy: 0.0779
Epoch 3/30
87/87 [==============================] - 10s 108ms/step - loss: 0.0808 - sparse_categorical_accuracy: 0.9770 - val_loss: 23.5948 - val_sparse_categorical_accuracy: 0.0455
Epoch 4/30
87/87 [==============================] - 10s 110ms/step - loss: 0.0762 - sparse_categorical_accuracy: 0.9773 - val_loss: 25.0027 - val_sparse_categorical_accuracy: 0.0357
Epoch 5/30
87/87 [==============================] - 10s 111ms/step - loss: 0.0731 - sparse_categorical_accuracy: 0.9795 - val_loss: 21.1595 - val_sparse_categorical_accuracy: 0.0357
Epoch 6/30
87/87 [==============================] - 11s 115ms/step - loss: 0.0672 - sparse_

In [15]:
# Training Model 3
history3 = model_3.fit(    
    train_dataset,
    epochs=30,
    shuffle=True,
    validation_data = (validation_dataset)
    )


Epoch 1/30
87/87 [==============================] - 34s 273ms/step - loss: 2.0481 - sparse_categorical_accuracy: 0.3917 - val_loss: 3.6334 - val_sparse_categorical_accuracy: 0.2468
Epoch 2/30
87/87 [==============================] - 13s 137ms/step - loss: 0.4149 - sparse_categorical_accuracy: 0.8651 - val_loss: 38.4023 - val_sparse_categorical_accuracy: 0.2208
Epoch 3/30
87/87 [==============================] - 13s 138ms/step - loss: 0.2008 - sparse_categorical_accuracy: 0.9450 - val_loss: 25.1020 - val_sparse_categorical_accuracy: 0.1461
Epoch 4/30
87/87 [==============================] - 13s 139ms/step - loss: 0.1757 - sparse_categorical_accuracy: 0.9478 - val_loss: 2.5688 - val_sparse_categorical_accuracy: 0.6006
Epoch 5/30
87/87 [==============================] - 13s 138ms/step - loss: 0.0963 - sparse_categorical_accuracy: 0.9737 - val_loss: 5.8490 - val_sparse_categorical_accuracy: 0.3604
Epoch 6/30
87/87 [==============================] - 13s 139ms/step - loss: 0.1112 - sparse_ca

# Save Model

In [29]:
modelFileDirectoryName1 = 'saved_model/Model_1'
modelFileDirectoryName2 = 'saved_model/Model_2'
modelFileDirectoryName3 = 'saved_model/Model_3'
model_1.save(modelFileDirectoryName1)
model_2.save(modelFileDirectoryName2)
model_3.save(modelFileDirectoryName3)

INFO:tensorflow:Assets written to: saved_model/Model_1\assets


INFO:tensorflow:Assets written to: saved_model/Model_1\assets


INFO:tensorflow:Assets written to: saved_model/Model_2\assets


INFO:tensorflow:Assets written to: saved_model/Model_2\assets


INFO:tensorflow:Assets written to: saved_model/Model_3\assets


INFO:tensorflow:Assets written to: saved_model/Model_3\assets


# Model Evaluation and History

In [36]:
# Model 1
loss, acc = model_1.evaluate(test_dataset)
print("Test accuracy: {:5.2f}%".format(100 * acc))

10/10 [==============================] - 1s 36ms/step - loss: 5.3646 - sparse_categorical_accuracy: 0.9583
Test accuracy: 95.83%


In [40]:
# Model 2
loss, acc = model_2.evaluate(test_dataset)
print("Test accuracy: {:5.2f}%".format(100 * acc))


10/10 [==============================] - 1s 18ms/step - loss: 33.6375 - sparse_categorical_accuracy: 0.0609
Test accuracy:  6.09%


In [39]:
# Model 3
loss, acc = model_3.evaluate(test_dataset)
print("Test accuracy: {:5.2f}%".format(100 * acc))

10/10 [==============================] - 1s 26ms/step - loss: 0.0051 - sparse_categorical_accuracy: 1.0000
Test accuracy: 100.00%
